### Year 1 Topology - Limit Checking - Overloading, Voltage limits

In [2]:
import pandas as pd

##### Loading

##### Percentage Loading in Branches 

Calculation related can be found in PSSE PAGVI Manual Chapter 6 Power system network simulations - Section 6.6.7 Activity Rate 

%I = MVA(actual)/(MVA(rated)*Vpu)

Vpu -> Bus Voltage at from end 

In [3]:
list_all = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for lsc in list_lsc:
            file_csv_out = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'_l.csv'
            data_load = pd.read_csv(file_csv_out,header=1, skiprows=0)
            # Extracted data contains branch loading from the metered and non-metered ends. The data is filtered to obtain the branch loading from the metered end.
            data_fil = data_load[data_load['From Bus']!=data_load['Non Metered Bus']].reset_index(drop=True)
            data_fil['From Bus V'] = data_fil['From Bus Ext'].str.extract(r'(\d+\.\d+)') 
            data_fil['To Bus V'] = data_fil['To Bus Ext'].str.extract(r'(\d+\.\d+)')
            data_fil['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
            list_all.append(data_fil)
data_all = pd.concat(list_all).reset_index(drop=True)
data_all = data_all[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID',
       'Non Metered Bus', ' Branch I', 'Branch Flow MW', 'Branch Flow MVAR',
       'Branch MVA', 'MW Loss', 'MVAR Loss', 'Rate', '%I', '%MVA',
       'From Bus V', 'To Bus V']]

##### Checking transformer loading greater than 80% 

In [4]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
output = transformer_loading[transformer_loading['%MVA'] > 80.0].reset_index(drop=True)
group_out = output.groupby('Scenario')
list_output_scenario = list(output['Scenario'].unique())
list_output_scenario
for list_out in list_output_scenario:
    display(group_out.get_group(list_out))

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
0,"Solar = 0 MW, Hydro = 400 MW, LLS",202,EAST500 500.00,203,EAST230 230.00,1,203,733,613.40,219.17,651.38,1.62,65.63,800.0,79.44,81.42,500.00,230.00
1,"Solar = 0 MW, Hydro = 400 MW, LLS",204,SUB500 500.00,205,SUB230 230.00,1,205,756,638.46,140.29,653.69,1.29,64.31,800.0,81.85,81.71,500.00,230.00


,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
2,"Solar = 0 MW, Hydro = 400 MW, RLS",202,EAST500 500.00,203,EAST230 230.00,1,203,767,632.20,238.77,675.79,1.77,71.75,800.0,83.06,84.47,500.00,230.00
3,"Solar = 0 MW, Hydro = 400 MW, RLS",204,SUB500 500.00,205,SUB230 230.00,1,205,785,654.33,160.11,673.63,1.39,69.46,800.0,85.06,84.20,500.00,230.00


,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
4,"Solar = 0 MW, Hydro = 400 MW, HLS",202,EAST500 500.00,203,EAST230 230.00,1,203,814,646.79,284.91,706.76,1.99,80.91,800.0,88.21,88.35,500.00,230.00
5,"Solar = 0 MW, Hydro = 400 MW, HLS",204,SUB500 500.00,205,SUB230 230.00,1,205,831,666.32,211.54,699.09,1.56,77.86,800.0,90.06,87.39,500.00,230.00


,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
6,"Solar = 0 MW, Hydro = 500 MW, LLS",202,EAST500 500.00,203,EAST230 230.00,1,203,796,670.74,216.08,704.68,1.90,77.28,800.0,86.20,88.09,500.00,230.00
7,"Solar = 0 MW, Hydro = 500 MW, LLS",204,SUB500 500.00,205,SUB230 230.00,1,205,844,717.23,125.52,728.13,1.61,80.31,800.0,91.46,91.02,500.00,230.00


,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
8,"Solar = 0 MW, Hydro = 500 MW, RLS",202,EAST500 500.00,203,EAST230 230.00,1,203,841,686.76,258.66,733.86,2.12,86.27,800.0,91.08,91.73,500.00,230.00
9,"Solar = 0 MW, Hydro = 500 MW, RLS",204,SUB500 500.00,205,SUB230 230.00,1,205,887,730.71,172.39,750.77,1.77,88.55,800.0,96.04,93.85,500.00,230.00


...

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
41,"Solar = 100 MW, Hydro = 500 MW, RLS",202,EAST500 500.00,203,EAST230 230.00,1,203,803,660.86,246.29,705.27,1.94,78.70,800.0,86.99,88.16,500.00,230.00
42,"Solar = 100 MW, Hydro = 500 MW, RLS",204,SUB500 500.00,205,SUB230 230.00,1,205,845,702.86,160.55,720.96,1.61,80.44,800.0,91.54,90.12,500.00,230.00


,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
43,"Solar = 100 MW, Hydro = 500 MW, HLS",202,EAST500 500.00,203,EAST230 230.00,1,203,851,675.54,291.71,735.84,2.17,88.33,800.0,92.16,91.98,500.00,230.00
44,"Solar = 100 MW, Hydro = 500 MW, HLS",204,SUB500 500.00,205,SUB230 230.00,1,205,891,714.78,210.64,745.17,1.79,89.42,800.0,96.51,93.15,500.00,230.00


,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
45,"Solar = 100 MW, Hydro = 600 MW, LLS",202,EAST500 500.00,203,EAST230 230.00,1,203,828,697.49,220.12,731.40,2.06,83.68,800.0,89.70,91.42,500.00,230.00
46,"Solar = 100 MW, Hydro = 600 MW, LLS",204,SUB500 500.00,205,SUB230 230.00,1,205,901,763.85,121.77,773.49,1.83,91.35,800.0,97.55,96.69,500.00,230.00


,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
47,"Solar = 100 MW, Hydro = 600 MW, RLS",202,EAST500 500.00,203,EAST230 230.00,1,203,882,715.23,273.07,765.59,2.34,95.00,800.0,95.57,95.70,500.00,230.00
48,"Solar = 100 MW, Hydro = 600 MW, RLS",204,SUB500 500.00,205,SUB230 230.00,1,205,952,778.53,179.91,799.04,2.04,102.08,800.0,103.12,99.88,500.00,230.00


,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V
49,"Solar = 100 MW, Hydro = 600 MW, HLS",202,EAST500 500.00,203,EAST230 230.00,1,203,934,729.65,320.44,796.91,2.62,106.42,800.0,101.16,99.61,500.00,230.00
50,"Solar = 100 MW, Hydro = 600 MW, HLS",204,SUB500 500.00,205,SUB230 230.00,1,205,1003,790.15,231.20,823.28,2.26,113.19,800.0,108.58,102.91,500.00,230.00


##### Checking branch loading greater than 80% 

In [26]:
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)
branch_overload = branch_loading[branch_loading['%I'] > 80.0].reset_index(drop=True)
branch_overload['Branch'] = branch_overload['From Bus'].astype(str) + ' ' + branch_overload['From Bus Ext'] + ' to ' + \
branch_overload['To Bus'].astype(str) + ' '+ branch_overload['To Bus Ext'] + ' ' + branch_overload['Ckt ID'].astype(str)
list_branch = branch_overload['Branch'].unique()
len(list_branch)
print(list_branch)
# for branch in list_branch:
#     load = list(branch_overload[branch_overload['Branch'] == branch])
#     display(len(load))


['153 MID230      230.00 to 154 DOWNTN      230.00 1'
 '3001 MINE        230.00 to 3003 S. MINE     230.00 1'
 '3005 WEST        230.00 to 3007 RURAL       230.00 1'
 '153 MID230      230.00 to 154 DOWNTN      230.00 2'
 '154 DOWNTN      230.00 to 203 EAST230     230.00 1']


##### Voltage Upper Limits 

In [6]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for lsc in list_lsc:
            file_sav_in = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'_volt.csv'
        data_volt = pd.read_csv(file_sav_in,header=1, skiprows=0)
        data_volt['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
        list_volt.append(data_volt)
data_all_volt = pd.concat(list_volt).reset_index(drop=True)
data_all_volt = data_all_volt[['Scenario','Bus Number', 'Bus Name', 'Area', 'Base Voltage', 'Bus Voltage(PU)','Bus Voltage(kV)']]

In [7]:
data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV)
0,"Solar = 0 MW, Hydro = 500 MW, HLS",211,HYDRO_G,2,20.0,1.061,21.228
1,"Solar = 0 MW, Hydro = 500 MW, HLS",212,HYDRO_N,2,20.0,1.061,21.228
2,"Solar = 0 MW, Hydro = 600 MW, HLS",211,HYDRO_G,2,20.0,1.080,21.609
3,"Solar = 0 MW, Hydro = 600 MW, HLS",212,HYDRO_N,2,20.0,1.080,21.609
4,"Solar = 50 MW, Hydro = 500 MW, HLS",211,HYDRO_G,2,20.0,1.057,21.131
5,"Solar = 50 MW, Hydro = 500 MW, HLS",212,HYDRO_N,2,20.0,1.057,21.131
6,"Solar = 50 MW, Hydro = 600 MW, HLS",211,HYDRO_G,2,20.0,1.075,21.495
7,"Solar = 50 MW, Hydro = 600 MW, HLS",212,HYDRO_N,2,20.0,1.075,21.495
8,"Solar = 100 MW, Hydro = 500 MW, HLS",211,HYDRO_G,2,20.0,1.053,21.053
9,"Solar = 100 MW, Hydro = 500 MW, HLS",212,HYDRO_N,2,20.0,1.053,21.053


##### Voltage Lower Limits

In [8]:
data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95]

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV)
6,"Solar = 0 MW, Hydro = 400 MW, HLS",154,DOWNTN,1,230.0,0.931,214.213
11,"Solar = 0 MW, Hydro = 400 MW, HLS",205,SUB230,2,230.0,0.941,216.472
27,"Solar = 0 MW, Hydro = 500 MW, HLS",103,SOLAR_PV,1,13.8,0.946,13.057
31,"Solar = 0 MW, Hydro = 500 MW, HLS",154,DOWNTN,1,230.0,0.916,210.790
34,"Solar = 0 MW, Hydro = 500 MW, HLS",203,EAST230,2,230.0,0.945,217.303
36,"Solar = 0 MW, Hydro = 500 MW, HLS",205,SUB230,2,230.0,0.926,213.079
46,"Solar = 0 MW, Hydro = 500 MW, HLS",3007,RURAL,5,230.0,0.945,217.277
47,"Solar = 0 MW, Hydro = 500 MW, HLS",3008,CATDOG,5,230.0,0.938,215.770
52,"Solar = 0 MW, Hydro = 600 MW, HLS",103,SOLAR_PV,1,13.8,0.925,12.766
56,"Solar = 0 MW, Hydro = 600 MW, HLS",154,DOWNTN,1,230.0,0.895,205.796


#### Conclusion 

Year 1 Toplogy limit violation check for overloading and voltage was carried out for the three load scenarios. It was seen that
1. overloading of greater than 80% was reported for following branches and transformers for considered solar output and load scenarios 
    * Solar = 0 MW, HLS \
      Branch 153 MID230 230.00 154 DOWNTN 230.00 Ckt ID 1
    * Solar = 0 MW, HLS	\
    Transformer 205	SUB230 230.00	206	URBGEN 18.000
    * Solar = 50 MW, HLS	\
    Transformer 205	SUB230 230.00	206	URBGEN 18.000
    * Solar = 100 MW, HLS	\
    Transformer 205	SUB230 230.00	206	URBGEN 18.000
2. The buses violating the upper voltage limit for various scenarios are:
    * Solar = 0 MW, Scenarios = LLS, RLS \
      BUS 201, 308
    * Solar = 0 MW, Scenarios = HLS

      BUS 201
    * Solar = 50 MW, Scenarios = LLS, RLS \
      BUS 201, 308
    * Solar = 50 MW, Scenarios = HLS \
      BUS 201
    * Solar = 100 MW, Scenarios = LLS, RLS \
      BUS 201, 308
    * Solar = 100 MW, Scenarios = HLS 
    
      BUS 201